In [ ]:
CALL apoc.load.json('file:///kakao.json') YIELD value
UNWIND keys(value) AS storeId
WITH storeId, value[storeId] AS storeData
MATCH (s:STORE {pk: toInteger(storeId)})
SET s.name = storeData.search_result_nm,
    s.rating = storeData.rating,
    s.rating_count = storeData.rating_count,
    s.image_url = storeData.image_url

WITH s, storeData, storeId
UNWIND keys(storeData.review) AS reviewKey
WITH s, reviewKey,storeId,storeData.review[reviewKey] AS reviewData
MERGE (r:Review {id: reviewKey,source: 'Kakao', storePk: toInteger(storeId)})
SET r:Kakao,
    r.text = reviewData.review,
    r.user_id = reviewData.user_id
MERGE (s)-[:HAS_REVIEW]->(r);